In [1]:
import warnings
import sys
import pathlib
import torch
import gpytorch
from IPython.display import HTML


warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2

import loaders.hrrr
import fields.continuous

In [2]:
import loaders.hrrr
import fields.discrete
import fields.continuous
import models.gp
import models.neural_flow

In [3]:
hrrr = loaders.hrrr.Data(date="2024-09-26 17:00", hours=4, level= 700)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws


In [4]:
dvf = hrrr.vector_field(extent=(-88,-81,24.5,29))
dsf = hrrr.scalar_field(extent=(-88,-81,24.5,29))

In [5]:
HTML(dsf.plot().to_html5_video())

In [41]:
cvf = fields.continuous.VectorField()

In [42]:
cvf.train_mle(dsf, dsf.coord_field.TXY.shape[0]//3, epochs = 50)

Training GP with MLE...


Epoch 1/50 — loss: 0.5332 — noise_var: 0.0752 — lengthscale: [3.706458   0.9184378  0.91826624]


Epoch 2/50 — loss: 0.4398 — noise_var: 0.0575 — lengthscale: [3.4135697 1.0990504 1.1044313]


Epoch 3/50 — loss: 0.4011 — noise_var: 0.0481 — lengthscale: [3.120604  1.2742553 1.3011613]


Epoch 4/50 — loss: 0.3714 — noise_var: 0.0453 — lengthscale: [2.8316686 1.4173992 1.502683 ]


...

Epoch 46/50 — loss: 0.2527 — noise_var: 0.0142 — lengthscale: [2.1841948 2.241325  2.3775897]


Epoch 47/50 — loss: 0.2491 — noise_var: 0.0141 — lengthscale: [2.1963565 2.2463553 2.3718908]


Epoch 48/50 — loss: 0.2496 — noise_var: 0.0140 — lengthscale: [2.2064414 2.2501826 2.3655522]


Epoch 49/50 — loss: 0.2461 — noise_var: 0.0140 — lengthscale: [2.2219946 2.2458045 2.3640926]


Epoch 50/50 — loss: 0.2480 — noise_var: 0.0139 — lengthscale: [2.2394257 2.238251  2.3668973]


In [43]:
HTML(cvf.plot(dvf.coord_field, factor=1).to_html5_video())

In [45]:
UV_hat = cvf.velocity(dvf.coord_field.TXY_scaled)*dvf.coord_field.XY_scale/ dvf.coord_field.T_scale
diff_sq = (UV_hat - dvf.UV).square()
diff_sq.mean(dim = -1).sqrt().mean()


tensor(4.7590, grad_fn=<MeanBackward0>)